## The Multidimensional Multiple Knapsack Problem (MMKP)

There is a set of items, each characterized by a value and a set of "weights" corresponding to different dimensions (e.g., physical weight, volume, or size).
We also have several knapsacks, each with its own capacity constraints in those same dimensions.

The objective is to assign items to knapsacks in order to **maximize the total value** of the selected items, while ensuring that no capacity constraint is violated.

## The Simulated Annealing Approach

To tackle this problem, i implemented a **Simulated Annealing (SA)** metaheuristic.
SA is inspired by the physical process of annealing, where a material is heated and then slowly cooled.
In the optimization context, this analogy translates into a gradual exploration of the solution space, where the algorithm can occasionally accept worse solutions to escape local optima, progressively reducing such acceptance as the “temperature” decreases.

### Algorithm Overview

1. **Initialization**
   The algorithm starts with a randomly generated solution. Each item is either:

   * assigned to one of the available knapsacks, or
   * left unassigned (represented by `-1`).
     The fitness of this initial configuration is evaluated using a custom evaluation function.

2. **Evaluation Function**
   The function `evaluate(solution)` computes:

   * the **total value** of the assigned items;
   * the **load** of each knapsack across all dimensions;
   * a **penalty** for constraint violations, proportional to the amount by which any capacity limit is exceeded.

3. **Neighborhood Generation**
   A new candidate solution is created by applying the `tweak()` function, which randomly modifies the current configuration.
   Specifically, one or more items (depending on a parameter `strength`) are reassigned to a different knapsack or marked as unassigned.
   This introduces controlled randomness to explore neighboring regions of the search space.

4. **Acceptance Criterion**
   Once the new solution is evaluated, it is compared with the current one:

   * If the new fitness is **higher**, the solution is always accepted.
   * If it is **lower**, it may still be accepted with a probability:  `e^(-Δf/T)`
   where (Δf) is the fitness loss and (T) is the current temperature. This probabilistic acceptance allows the algorithm to escape local optima during early stages.

5. **Cooling Schedule**
The temperature decreases gradually as the search progresses.
Every 200 iterations, the temperature is multiplied by a coefficient.
This cooling schedule balances exploration (at high temperatures) and exploitation (at low temperatures).

5. **Best Solution Tracking**
   Throughout the optimization process, any **feasible** solution (with zero penalty) that outperforms the previous best is recorded as the *best valid solution*.

6. **Termination**
   After a fixed number of iterations (`MAX_STEPS`), the algorithm returns the best feasible solution found, along with its total value, penalty, fitness, and performance ratio with respect to the theoretical maximum achievable value.

---

## Algorithm Flow Summary

```
Generate random initial solution
↓
Evaluate initial fitness
↓
Repeat for MAX_STEPS iterations:
    ↓
    Generate new candidate (tweak)
    ↓
    Evaluate candidate fitness
    ↓
    If better → accept
    If worse → accept with probability e^(-Δf/T)
    ↓
    Gradually decrease temperature
↓
Return the best feasible solution found
```

#Problem 1

In [ ]:
import numpy as np

# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [ ]:
import numpy as np
import random
from copy import deepcopy
from tqdm.auto import tqdm

# Evaluation function
def evaluate(solution):
    total_value = 0
    used_capacity = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)

    for item in range(len(solution)):
        sack = solution[item]
        if sack == -1:
            continue
        total_value += VALUES[item]
        used_capacity[sack] += WEIGHTS[item]

    # Penalty if constraints are violated
    penalty = 0
    for k in range(NUM_KNAPSACKS):
        for d in range(NUM_DIMENSIONS):
            if used_capacity[k, d] > CONSTRAINTS[k, d]:
                penalty += (used_capacity[k, d] - CONSTRAINTS[k, d])

    fitness = total_value - 10 * penalty
    return total_value, penalty, fitness


# Tweak function
def tweak(solution, strength=0.1):
    new_sol = deepcopy(solution)
    again = True
    while again:
        item = random.randint(0, NUM_ITEMS - 1)
        new_sol[item] = random.choice(list(range(NUM_KNAPSACKS)) + [-1])
        again = random.random() < strength
    return new_sol


# Simulated Annealing

# Generate random initial solution
current_solution = []
for _ in range(NUM_ITEMS):
    possible_choices = list(range(NUM_KNAPSACKS)) + [-1]
    chosen_sack = random.choice(possible_choices)
    current_solution.append(chosen_sack)

_, _, current_fitness = evaluate(current_solution)

MAX_STEPS = 100000
temp = 80.0

valid_solution = None
best_valid_value = 0
best_valid_fitness = 0

# Theoretical maximum total value of the instance of the problem
max_value = np.sum(VALUES)
print(f"Theoretical maximum total value: {max_value}\n")

for step in tqdm(range(MAX_STEPS)):
    val, pen, fit = evaluate(current_solution)

    if pen == 0 and fit > best_valid_fitness:
        valid_solution = deepcopy(current_solution)
        best_valid_value = val
        best_valid_fitness = fit
        ratio = (best_valid_value / max_value) * 100
        print(f"[Step {step}] New feasible solution found! "
              f"Value = {val}, Fitness = {fit}, Ratio = {ratio:.2f}% of max")

    # Cooling temperature
    if step % 200 == 0:
        temp *= 0.99

    # Generate new candidate solution
    new_solution = tweak(current_solution, strength=0.1)
    _, _, new_fitness = evaluate(new_solution)

    # Accept or reject new solution
    if new_fitness >= current_fitness:
        current_solution = new_solution
        current_fitness = new_fitness
    else:
        diff = current_fitness - new_fitness
        p = np.exp(-diff / temp)
        if random.random() < p:
            current_solution = new_solution
            current_fitness = new_fitness


# Output
if valid_solution is not None:
    val_b, pen_b, fit_b = evaluate(valid_solution)
    ratio_b = (val_b / max_value) * 100
    print(f"\nBest feasible solution found:")
    print(f"Value = {val_b}, Penalty = {pen_b}, Fitness = {fit_b}")
    print(f"Ratio = {ratio_b:.2f}% of theoretical maximum value")

    print("\nSolution array form:")
    print(np.array(valid_solution))

else:
    print("\nNo feasible solution found.")


Theoretical maximum total value: 1065



  0%|          | 0/100000 [00:00<?, ?it/s]

[Step 17] New feasible solution found! Value = 744, Fitness = 744, Ratio = 69.86% of max
[Step 18] New feasible solution found! Value = 813, Fitness = 813, Ratio = 76.34% of max
[Step 36] New feasible solution found! Value = 874, Fitness = 874, Ratio = 82.07% of max
[Step 46] New feasible solution found! Value = 882, Fitness = 882, Ratio = 82.82% of max
[Step 52] New feasible solution found! Value = 924, Fitness = 924, Ratio = 86.76% of max
[Step 76] New feasible solution found! Value = 956, Fitness = 956, Ratio = 89.77% of max
[Step 210] New feasible solution found! Value = 990, Fitness = 990, Ratio = 92.96% of max
[Step 234] New feasible solution found! Value = 993, Fitness = 993, Ratio = 93.24% of max
[Step 455] New feasible solution found! Value = 1020, Fitness = 1020, Ratio = 95.77% of max
[Step 615] New feasible solution found! Value = 1045, Fitness = 1045, Ratio = 98.12% of max
[Step 616] New feasible solution found! Value = 1057, Fitness = 1057, Ratio = 99.25% of max
[Step 5200

#Problem 2

In [ ]:
import numpy as np

rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [ ]:
import numpy as np
import random
from copy import deepcopy
from tqdm.auto import tqdm

# Evaluation function
def evaluate(solution):
    total_value = 0
    used_capacity = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)

    for item in range(len(solution)):
        sack = solution[item]
        if sack == -1:
            continue
        total_value += VALUES[item]
        used_capacity[sack] += WEIGHTS[item]

    # Penalty if constraints are violated
    penalty = 0
    for k in range(NUM_KNAPSACKS):
        for d in range(NUM_DIMENSIONS):
            if used_capacity[k, d] > CONSTRAINTS[k, d]:
                penalty += (used_capacity[k, d] - CONSTRAINTS[k, d])

    fitness = total_value - 10 * penalty
    return total_value, penalty, fitness


# Tweak function
def tweak(solution, strength=0.1):
    new_sol = deepcopy(solution)
    again = True
    while again:
        item = random.randint(0, NUM_ITEMS - 1)
        new_sol[item] = random.choice(list(range(NUM_KNAPSACKS)) + [-1])
        again = random.random() < strength
    return new_sol


# Simulated Annealing

# Generate random initial solution
current_solution = []
for _ in range(NUM_ITEMS):
    possible_choices = list(range(NUM_KNAPSACKS)) + [-1]
    chosen_sack = random.choice(possible_choices)
    current_solution.append(chosen_sack)

_, _, current_fitness = evaluate(current_solution)

MAX_STEPS = 100000
temp = 80.0

valid_solution = None
best_valid_value = 0
best_valid_fitness = 0

# Theoretical maximum total value of the instance of the problem
max_value = np.sum(VALUES)
print(f"Theoretical maximum total value: {max_value}\n")

for step in tqdm(range(MAX_STEPS)):
    val, pen, fit = evaluate(current_solution)

    if pen == 0 and fit > best_valid_fitness:
        valid_solution = deepcopy(current_solution)
        best_valid_value = val
        best_valid_fitness = fit
        ratio = (best_valid_value / max_value) * 100
        print(f"[Step {step}] New feasible solution found! "
              f"Value = {val}, Fitness = {fit}, Ratio = {ratio:.2f}% of max")

    # Cooling temperature
    if step % 200 == 0:
        temp *= 0.9995

    # Generate new candidate solution
    new_solution = tweak(current_solution, strength=0.1)
    _, _, new_fitness = evaluate(new_solution)

    # Accept or reject new solution
    if new_fitness >= current_fitness:
        current_solution = new_solution
        current_fitness = new_fitness
    else:
        diff = current_fitness - new_fitness
        p = np.exp(-diff / temp)
        if random.random() < p:
            current_solution = new_solution
            current_fitness = new_fitness


# Output
if valid_solution is not None:
    val_b, pen_b, fit_b = evaluate(valid_solution)
    ratio_b = (val_b / max_value) * 100
    print(f"\nBest feasible solution found:")
    print(f"Value = {val_b}, Penalty = {pen_b}, Fitness = {fit_b}")
    print(f"Ratio = {ratio_b:.2f}% of theoretical maximum value")

    print("\nSolution array form:")
    print(np.array(valid_solution))

else:
    print("\nNo feasible solution found.")


Theoretical maximum total value: 52620



  0%|          | 0/100000 [00:00<?, ?it/s]

[Step 1671] New feasible solution found! Value = 39290, Fitness = 39290, Ratio = 74.67% of max
[Step 1686] New feasible solution found! Value = 39653, Fitness = 39653, Ratio = 75.36% of max
[Step 3191] New feasible solution found! Value = 42162, Fitness = 42162, Ratio = 80.13% of max
[Step 3200] New feasible solution found! Value = 42290, Fitness = 42290, Ratio = 80.37% of max
[Step 4909] New feasible solution found! Value = 42333, Fitness = 42333, Ratio = 80.45% of max
[Step 5983] New feasible solution found! Value = 43091, Fitness = 43091, Ratio = 81.89% of max
[Step 10175] New feasible solution found! Value = 44756, Fitness = 44756, Ratio = 85.06% of max
[Step 13534] New feasible solution found! Value = 45678, Fitness = 45678, Ratio = 86.81% of max
[Step 15483] New feasible solution found! Value = 45739, Fitness = 45739, Ratio = 86.92% of max
[Step 17574] New feasible solution found! Value = 46419, Fitness = 46419, Ratio = 88.22% of max
[Step 17585] New feasible solution found! Valu

# Addressing the Dimensionality Feasibility Problem

The previous version of the **Simulated Annealing (SA)** algorithm often generated **infeasible solutions** (especially in high dimensions), violating one or more knapsack capacity constraints.
To overcome this limitation, the algorithm was enhanced with two new mechanisms:

* a **constructive initialization** procedure `build_feasible_solution()` that guarantees a valid starting point;
* a **feasibility repair mechanism** embedded in the `tweak()` function, ensuring that constraint violations are corrected after each modification.

Only the key differences are shown in this preview.

## Initialization `build_feasible_solution()`

Instead of starting from a random configuration, the algorithm constructs an initial **feasible solution** using a **greedy heuristic**.

1. Items are sorted by their **value-to-weight ratio**: `ratio_i = (value_i) / ∑d (weight_i,d)`


2. Each item is placed into the **first knapsack** where it fits without exceeding any constraint.
3. If no knapsack can accommodate it, the item remains **unassigned** (`-1`).

This guarantees that the algorithm begins with a **valid** configuration.

## Tweak Function with Feasibility Repair

The `tweak()` function explores the neighborhood of the current solution by **randomly reassigning** items to different knapsacks or leaving them unassigned.

After the perturbation, the algorithm automatically repairs any constraint violations to restore feasibility.

**Repair procedure:**

1. Compute the current loads for each knapsack.
2. For every knapsack that exceeds a capacity constraint:

   * Identify all items assigned to it;
   * Compute each item’s **value density**: `density_i ​= value_i​​ / ∑d ​weight_i,d​`
     
   * Iteratively remove the **least efficient** (lowest-density) items until all constraints are satisfied.

## Hill Climbing Optimization Strategy

After ensuring that a feasible solution is always produced, SA loose part of its potential. So i adopted a simple Hill Climbing approach.

Core Idea

At each iteration, a neighboring solution is generated through the tweak() operator.
This operator introduces small random changes (adding, removing, or swapping items between knapsacks) while preserving feasibility.

If the new configuration yields an equal or higher total value, it is immediately accepted as the new current solution.
Otherwise, the algorithm discards it and continues exploring other neighbors.

#Problem 3

In [6]:
import numpy as np

# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

##Accept dense-value feasible solution

In [16]:
import numpy as np
import random
from copy import deepcopy
from tqdm.auto import tqdm

# Probability for each tweak operation
P_REMOVE = 0.3
P_ADD = 0.3
P_SWAP = 0.4

def evaluate(solution):
    total_value = 0
    for item in range(len(solution)):
        sack = solution[item]
        if sack != -1:
            total_value += VALUES[item]
    return total_value

def tweak(solution, strength=0.1):
    new_sol = deepcopy(solution)

    while True:
        op_choice = random.random()

        # Remove an item
        if op_choice < P_REMOVE:
            assigned_items = [i for i, sack in enumerate(new_sol) if sack != -1]
            if assigned_items:
                item = random.choice(assigned_items)
                new_sol[item] = -1

        # Add an item
        elif op_choice < P_REMOVE + P_ADD:
            unassigned_items = [i for i, sack in enumerate(new_sol) if sack == -1]
            if unassigned_items:
                item = random.choice(unassigned_items)
                new_sol[item] = random.randint(0, NUM_KNAPSACKS - 1)

        # Swap items between knapsacks
        else:
            assigned_items = [i for i, sack in enumerate(new_sol) if sack != -1]
            if len(assigned_items) >= 2:
                i1, i2 = random.sample(assigned_items, 2)
                sack1, sack2 = new_sol[i1], new_sol[i2]
                new_sol[i1], new_sol[i2] = sack2, sack1

        # Repeat tweak with a given probability (strength)
        if random.random() >= strength:
            break

    # Feasibility correction
    loads = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
    for i, sack in enumerate(new_sol):
        if sack != -1:
            loads[sack] += WEIGHTS[i]

    for k in range(NUM_KNAPSACKS):
        while np.any(loads[k] > CONSTRAINTS[k]):
            items_in_k = [i for i, s in enumerate(new_sol) if s == k]
            if not items_in_k:
                break
            densities = [(i, VALUES[i] / (np.sum(WEIGHTS[i]) + 1e-6)) for i in items_in_k]
            worst_item = min(densities, key=lambda x: x[1])[0]
            new_sol[worst_item] = -1
            loads[k] -= WEIGHTS[worst_item]

    return new_sol


# Generate an initial feasible solution
def build_feasible_solution()():
    solution = [-1] * NUM_ITEMS
    loads = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)

    # Compute density for each item
    densities = []
    for i in range(NUM_ITEMS):
        avg_weight = np.mean(WEIGHTS[i])
        density = VALUES[i] / (avg_weight + 1e-6)
        densities.append((i, density))

    densities.sort(key=lambda x: x[1], reverse=True)

    # Assign each item to the first knapsack that has enough capacity
    for item_idx, _ in densities:
        item_weight = WEIGHTS[item_idx]

        for k in range(NUM_KNAPSACKS):
            if np.all(loads[k] + item_weight <= CONSTRAINTS[k]):
                solution[item_idx] = k
                loads[k] += item_weight
                break

    return solution


print("Generating initial feasible solution...")
current_solution = build_feasible_solution()()

initial_value = evaluate(current_solution)
print(f"Initial solution: Value = {initial_value}")
current_value = initial_value

MAX_STEPS = 50000

best_solution = deepcopy(current_solution)
best_value = initial_value

# Compute theoretical maximum possible value
max_possible_value = np.sum(VALUES)
best_ratio = (best_value / max_possible_value) * 100

print(f"Theoretical maximum total value: {max_possible_value}\n")


pbar = tqdm(range(MAX_STEPS), desc="Hill Climbing", unit="step")

for step in pbar:
    new_solution = tweak(current_solution, strength=0.3)
    new_value = evaluate(new_solution)

    if new_value >= current_value:
        current_solution = new_solution
        current_value = new_value

        if current_value > best_value:
            best_solution = deepcopy(current_solution)
            best_value = current_value
            best_ratio = (best_value / max_possible_value) * 100

    pbar.set_postfix({
        "Best Value": f"{best_value}",
        "Ratio": f"{best_ratio:.2f}%"
    })

pbar.close()

# Output
ratio_b = (best_value / max_possible_value) * 100
print(f"\nBest solution found:")
print(f"Value = {best_value}")
print(f"Ratio = {ratio_b:.2f}% of theoretical maximum value")


Generating initial feasible solution...
Initial solution: Value = 1818128
Theoretical maximum total value: 2490698



Hill Climbing:   0%|          | 0/50000 [00:00<?, ?step/s]


Best solution found:
Value = 1992320
Ratio = 79.99% of theoretical maximum value
